In [6]:
import numpy as np
import keras
from keras.layers.core import Dense, Activation, Dropout,Reshape
from keras.models import Sequential
from keras.optimizers import SGD,Adam,Adadelta
from keras import regularizers
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

def rf():
    param_test1 = {'n_estimators':[10, 30, 50, 70]}
    clf = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt' ,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',cv=5)
    return clf
def panel_append(p):
    result = []
    r = np.concatenate((np.r_[1:21],np.r_[40:241:20]))
    for m in r:
        rt = (p/p.shift(m)) - 1
        result.append(rt)
    return list(np.array(result).T)
def get_profit_shenzhi(start_index, end_index, cost = 0,k = 5):
    profit_all, profit_everyday, accuracy_train = [],[],[]
    index = []
    stds, hit_ratios, sharp_ratios = [],[],[]
    ms = []
    
    for i in np.r_[start_index:end_index:250]:
# Get close prices
        ics = index_components('399001.XSHE')
        p = get_price(ics, '20050104','20180709',frequency = '1d')['close'][i:i+1000].dropna(axis = 'columns', how = 'any')
        dates = p.index
        ics_new = p.columns
        if (len(ics_new) < k*2):
            print(len(ics_new))
            continue
        rt = (p - p.shift(1))/p
        med = rt.median(axis = 1)
        x_all = p.apply(lambda z : panel_append(z))
        X_train = []
        y_train = []
        for ic in ics_new:
            x = x_all[ic]
            y = (rt[ic] > med) * 1
            X_train += list(x)[241:749]
            y_train += list(y)[242:750]
        if (len(X_train) != len(y_train)):
            print('i:',i)
            continue
        model_rf = rf()
        model_rf.fit(X_train, y_train)
        pred_train = model_rf.predict(X_train)
        acc_train = accuracy_score(y_train, pred_train)
        dic = pd.DataFrame()
        for ic in ics_new:
            y = (rt[ic] > med) * 1
            X_test = list(x_all[ic])[750: -1]   
            proba_test = model_rf.predict_proba(X_test)
            dic[ic] = np.array(proba_test)[:,1]
        profit = 0
        ed_profit =  []
        if (len(dic.index) < 248):
            print(len(dic.index))
            continue
        for t in np.r_[750:999]:
            top_ks = dic.sort_values(by = dic.index[t-750], axis = 'columns', ascending = False).columns[:k]
            this_profit = 0
            for ic in top_ks:
                this_profit += rt[ic][t+1]
            profit += this_profit/k
            ed_profit.append(this_profit/k)
        profit_all.append(profit)
        hit_ratio = (len(np.array(ed_profit)[np.array(ed_profit) > 0] )/len(ed_profit))
        hit_ratios.append(hit_ratio)
        profit_everyday.append(ed_profit)
        accuracy_train.append(acc_train)
        index.append(i)
        std = np.std(ed_profit)
        m = np.average(ed_profit)
        ms.append(m)
        stds.append(std)
        sharp_ratios.append((m * np.sqrt(250))/std)
        print("train accuracy is: ", acc_train,"profit: ", profit, " hit ratio: ", hit_ratio, ' sharp ratio: ', (m * np.sqrt(250))/std,' daily return: ', m)
# Construct a dataframe
    df = pd.DataFrame(index = index)
    df['profits'] = profit_all
    df['everyday profit'] = profit_everyday
    df['training accuracy'] = accuracy_train
    df['standard deviation'] = stds
    df['hit ratio'] = hit_ratios
    df['sharp ratio'] = sharp_ratios
    df['daily return'] = ms
    return df

In [3]:
df = get_profit_shenzhi(0, 1000)

NameError: name 'index_components' is not defined

# logistic Regression

In [5]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer
import pandas as pd

In [ ]:

def panel_append(p):
    result = []
    r = np.concatenate((np.r_[1:21],np.r_[40:241:20]))
    for m in r:
        rt = (p/p.shift(m)) - 1
        result.append(rt)
    return list(np.array(result).T)
def Standard_error(sample):
    std=np.std(sample,ddof=0)
    standard_error=std/math.sqrt(len(sample))
    return standard_error

def get_profit(start_index, end_index, cost = 0,k = 5):
    profit_all, profit_everyday, accuracy_train = [],[],[]
    index = []
    returns,dates = [],[]
    ics = index_components('399001.XSHE')
    ps = get_price(ics, '20050104','20180709',frequency = '1d')['close']
    for i in np.r_[start_index:end_index:250]:
# Get close prices
        p = ps[i:i+1000].dropna(axis = 'columns', how = 'any')
        ics_new = p.columns
        if (len(ics_new) < k*2):
            print(len(ics_new))
            continue
        rt = (p - p.shift(1))/p
        med = rt.median(axis = 1)
        x_all = p.apply(lambda z : panel_append(z))
        X_train = []
        y_train = []
        for ic in ics_new:
            x = x_all[ic]
            y = (rt[ic] > med) * 1
            X_train += list(x)[241:749]
            y_train += list(y)[242:750]
        if (len(X_train) != len(y_train)):
            print('i:',i)
            continue
        model_log = LogisticRegression()
        model_log.fit(X_train, y_train)
        pred_train = model_log.predict(X_train)
        acc_train = accuracy_score(y_train, pred_train)
        dic = pd.DataFrame()
        for ic in ics_new:
            y = (rt[ic] > med) * 1
            X_test = list(x_all[ic])[750: -1]   
            proba_test = model_log.predict_proba(X_test)
            dic[ic] = np.array(proba_test)[:,1]
        if (len(dic.index) < 248):
            continue
        for t in np.r_[750:999]:
            top_ks = dic.sort_values(by = dic.index[t-750], axis = 'columns', ascending = False).columns[:k]
            this_profit = 0
            for ic in top_ks:
                this_profit += rt[ic][t+1]
#             print(this_profit/k, list(ps.index)[i+t])
            returns.append(this_profit/k)
            dates.append(ps.index[i+t])
            
    return returns, dates

In [ ]:
returns,dates = get_profit(0, 3000)
df = pd.DataFrame(returns, index = dates)

In [ ]:

m = np.average(returns)
std = np.std(returns)
ste = std/np.sqrt(len(returns))
t_stat = m/ste
median = np.median(returns)
q1 = np.percentile(returns,25)
q3 = np.percentile(returns,75)
hit_ratio = len(np.array(returns)[np.array(returns) > 0] )/len(returns)
maximum = max(returns)
minimum = min(returns)
skew = df.skew()[0]
kurt = df.kurt()[0]
log_stats_a = [m,ste,t_stat,minimum,q1,median,q3,maximum,hit_ratio,std,skew,kurt]
label_a_1 = ['Mean return','Standard error','t-statistic','Minimum','Quartile 1','Median','Quartile 3']
label_a_2 = ['Maximum','Share > 0','Standard dev.','Skewness','Kurtosis']
label_a = label_a_1 + label_a_2
panel_A = pd.DataFrame(index = label_a)
panel_A['LOG'] = log_stats_a

In [ ]:
panel_A

In [ ]:
from scipy.stats import norm

In [ ]:
VaR1 = norm.ppf(0.01, m, std)
VaR5 = norm.ppf(0.05, m, std)